<h1 style="text-align:center;">Reverting the BIOS to it's default values</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contains a example code to demonstrate how to revert BIOS to it's default values. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
        <li>getpass - to take password as inputs securely</li> 
    </ul>
</div>

In [ ]:
# Import the required libraries
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

# Import the helper function
from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our reset bios to default function overview, in the end this will set the bios settings to default.</h3><br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>

1. In the <b>bios_revert_default</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. In the ILO rest api the information about bios is stored inside the <b>systems json object</b>.
    And now through system_response variable,        we are making a HTTP <a href = "https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods">GET operation</a> and trying to obtain the bios settings of iLO system.
    <br>

3. In the system members response object with different <b>odata_id</b> we get a bios uri .
    Then again an HTTP GET request is made and response is stored in <b>bios_response</b>.
    Then we set the resret_action_uri.
    <br>

4. Else i.e; resource directory is enabled then we will simply iterate through all instances inside resources 
    and if bios attribute is present in <b>@odata.type</b> attribute then we'll set bios_uri and <i> get </i> bios settings of the user through redfish object and then then set the path to post the action we will perform
    <br>

5. Then we set the bios settings to default using the action type as <i>Bios.ResetBios</i> and say set it to <i>default</i>

    Ultimately if the operation was successfull without any errors then get the Success. If not we get the occured error.

 </div>

In [ ]:
def bios_revert_default(_redfishobj):
    # Initialize the BIOS reset action URI (Uniform Resource Identifier)
    bios_reset_action_uri = None
    
    # Get the resource directory to find the relevant URI
    resource_instances = get_resource_directory(_redfishobj)

    # If resource directory is not available or not to be used, find the relevant URI
    if DISABLE_RESOURCE_DIR or not resource_instances:
        # Get the systems URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        
        # Get the systems members URI
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        
        # Get the BIOS URI
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
        bios_response = _redfishobj.get(bios_uri)
        
        # Get the BIOS reset action URI
        bios_reset_action_uri = bios_response.obj['Actions']['#Bios.ResetBios']['target']
    else:
        # Use resource directory to find the relevant URI
        for instance in resource_instances:
            if '#Bios.' in instance['@odata.type']:
                bios_uri = instance['@odata.id']
                bios_data = _redfishobj.get(bios_uri)
                bios_reset_action_uri = bios_data.obj['Actions']['#Bios.ResetBios']['target']
                break
    
    # Define the body for the POST request
    body = {'Action': 'Bios.ResetBios', 'ResetType': 'default'}
    
    # Send the POST request
    resp = _redfishobj.post(bios_reset_action_uri, body)
    
    # Check the response status
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> class takes 3 parameters<i> iLO URL, Username, Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
    <br>

4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
    <br>
5. Then we call <b>bios_revert_default</b> with our redfish object and rest the settings to default.<br>
    

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    # Call the function to revert BIOS to default values
    bios_revert_default(REDFISHOBJ)

    # Logout from the Redfish client
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help please check the follwoing links for reference:
    <br>
    1. Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">Python ilorest library</a><br>
    2. HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">ilorestful api explorer</a><br>
    3. HPE iLO Redfish Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/">HPE iLO Redfish API Documentation</a><br>
    4. REST API: <a href = "https://restfulapi.net/">Restful API</a><br>    
</div>